In [225]:
import requests
import pandas as pd
import json
import sqlite3

In [226]:
#Install PyDrive2
!pip install -U -q PyDrive2
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#GoogleAuth
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [232]:
#file downloaded to your memory
file_download = drive.CreateFile({'id':'1dF_b1W_KEQr64zlACNiRhJh6qgo1-zae'})
file_download.GetContentFile('api_mst_tbl.csv')
df = pd.read_csv('api_mst_tbl.csv', delimiter=',')

v_tbl_nm = df.API_Name
v_endpoint = df.API_Endpoint

df.head()

,API_Id,API_Name,API_Description,API_Endpoint
0,API_0001,Historical_Dividends,A list of historical dividend payments for pub...,https://financialmodelingprep.com/api/v3/histo...
1,API_0002,Delisted_Companies,The FMP Delisted Companies endpoint provides a...,https://financialmodelingprep.com/api/v3/delis...


In [228]:
#Insert data that you requested to table
api_key = 'RpFio3zgpX4yA5A1j7Dw91lufq99xA9I'

for ind in v_endpoint.index:
  url = v_endpoint.loc[ind]
  req = requests.get(url+"?apikey="+api_key)
  # Convert JSON to a Python dictionary
  json_data = req.json()
  # Convert the dictionary to a pandas DataFrame
  df = pd.DataFrame(json_data)

  df_mapsql = df.applymap(str)

  # Connect to an SQLite database (create a new database if it doesn't exist)
  conn = sqlite3.connect('ttw_test.db')

  # Insert the DataFrame into an SQLite table
  df_mapsql.to_sql(v_tbl_nm[ind], conn, if_exists='replace', index=False)

  # Query the table to verify the insertion
  read_sql = pd.read_sql('SELECT * FROM '+v_tbl_nm[ind], conn)

  # Display the result
  print(read_sql)
  #read_sql
  # Close the database connection
  conn.close()

   symbol                                         historical
0    AAPL  {'date': '2023-11-10', 'label': 'November 10, ...
1    AAPL  {'date': '2023-08-11', 'label': 'August 11, 23...
2    AAPL  {'date': '2023-05-12', 'label': 'May 12, 23', ...
3    AAPL  {'date': '2023-02-10', 'label': 'February 10, ...
4    AAPL  {'date': '2022-11-04', 'label': 'November 04, ...
..    ...                                                ...
76   AAPL  {'date': '1988-05-16', 'label': 'May 16, 88', ...
77   AAPL  {'date': '1988-02-12', 'label': 'February 12, ...
78   AAPL  {'date': '1987-11-17', 'label': 'November 17, ...
79   AAPL  {'date': '1987-08-10', 'label': 'August 10, 87...
80   AAPL  {'date': '1987-05-11', 'label': 'May 11, 87', ...

[81 rows x 2 columns]
   symbol                                   companyName exchange     ipoDate  \
0    APTM        Alpha Partners Technology Merger Corp.   NASDAQ  2023-12-28   
1    SABA   Saba Capital Income & Opportunities Fund II     NYSE  2024-01-05   
2    